# reference
http://www.kaggle.com/meaninglesslives/unet-resnet34-in-keras
https://www.kaggle.com/shaojiaxin/u-net-with-simple-resnet-blocks-v2-new-loss


In [6]:
import numpy as np
import pandas as pd
import six

from random import randint

import matplotlib.pyplot as plt
import seaborn  as sns

from skimage.transform import resize
from keras import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from keras.layers import Input, Dropout, BatchNormalization, Activation, Add,\
Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, UpSampling2D
from keras.preprocessing.image import load_img
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm_notebook
from keras import initializers
from keras import regularizers
from keras import constraints
from keras.utils import conv_utils
from keras.utils.data_utils import get_file
from keras.engine import InputSpec
from keras import backend as K
from keras.regularizers import l2
from keras import optimizers

from keras.engine.topology import Input
from keras.engine.training import Model
from keras.layers.convolutional import Conv2D, UpSampling2D, Conv2DTranspose
from keras.layers.core import Activation, SpatialDropout2D
from keras.layers.merge import concatenate, add
from keras import backend as K

import tensorflow as tf

from sklearn.cross_validation import train_test_split, StratifiedKFold
from sklearn.cross_validation import KFold

from glob import glob
from tqdm import tqdm
%matplotlib inline

import train_util
import util
from image_generator import Generator
%load_ext autoreload
%autoreload 2

K.set_image_dim_ordering('tf')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Build U-net model

In [7]:
def BatchActivate(x):
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def convolution_block(x, filters, size, strides=(1, 1), padding='same', activation=True):
    x = Conv2D(filters, size, strides=strides, padding=padding)(x)
    if activation == True:
        x = BatchActivate(x)
    return x

def residual_block(blockInput, num_filters=16, batch_activate = False):
    x = BatchActivate(blockInput)
    x = convolution_block(x, num_filters, (3, 3))
    x = convolution_block(x, num_filters, (3, 3), activation=False)
    x = Add()([x, blockInput])
    if batch_activate:
        x = BatchActivate(x)
    return x

In [8]:
size = (3, 3)
def build_model(input_layer, start_neurons, DropoutRatio = 0.5):
    #101 -> 50
    conv1 = Conv2D(start_neurons * 1, size, activation='relu', padding="same")(input_layer)
    conv1 = residual_block(conv1, start_neurons * 1)
    conv1 = residual_block(conv1, start_neurons * 1, True)
    pool1 = MaxPooling2D((2, 2))(conv1)
    pool1 = Dropout(DropoutRatio/2)(pool1)
    
    #50 -> 25
    conv2 = Conv2D(start_neurons * 2, size, activation=None, padding="same")(pool1)
    conv2 = residual_block(conv2, start_neurons * 2)
    conv2 = residual_block(conv2, start_neurons * 2, True)
    pool2 = MaxPooling2D((2, 2))(conv2)
    pool2 = Dropout(DropoutRatio)(pool2)
    
    # 25 -> 12
    conv3  = Conv2D(start_neurons * 4, size, activation=None, padding="same")(pool2)
    conv3 = residual_block(conv3, start_neurons * 4)
    conv3 = residual_block(conv3, start_neurons * 4, True)
    pool3 = MaxPooling2D((2, 2))(conv3)
    pool3 = Dropout(DropoutRatio)(pool3)
    
    #12 -> 6
    conv4 = Conv2D(start_neurons * 8, size, activation=None, padding='same')(pool3)
    conv4 = residual_block(conv4, start_neurons * 8)
    conv4 = residual_block(conv4, start_neurons * 8, True)
    pool4 = MaxPooling2D((2, 2))(conv4)
    pool4 = Dropout(DropoutRatio)(pool4)
    
    # Middle
    convm = Conv2D(start_neurons * 16, size, activation=None, padding="same")(pool4)
    convm = residual_block(convm, start_neurons * 16)
    convm = residual_block(convm, start_neurons * 16, True)
    
    # 6 -> 12
    deconv4 = Conv2DTranspose(start_neurons * 8, size, strides=(2, 2), padding="same")(convm)
    uconv4 = concatenate([deconv4, conv4])
    uconv4 = Dropout(DropoutRatio)(uconv4)
    uconv4 = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(uconv4)
    uconv4 = residual_block(uconv4, start_neurons * 8)
    uconv4 = residual_block(uconv4, start_neurons * 8, True)
    
    # 12 -> 25
    deconv3 = Conv2DTranspose(start_neurons * 4, size, strides=(2, 2), padding="valid")(uconv4)
    uconv3 = concatenate([deconv3, conv3])
    uconv3 = Dropout(DropoutRatio)(uconv3)
    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(uconv3)
    uconv3 = residual_block(uconv3, start_neurons * 4)
    uconv3 = residual_block(uconv3, start_neurons * 4, True)
    
    # 25 -> 50
    deconv2 = Conv2DTranspose(start_neurons * 2, size, strides=(2, 2), padding="same")(uconv3)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = Dropout(DropoutRatio)(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(uconv2)
    uconv2 = residual_block(uconv2, start_neurons * 2)
    uconv2 = residual_block(uconv2, start_neurons * 2, True)
    
    #50 -> 101
    deconv1 = Conv2DTranspose(start_neurons * 1, size, strides=(2, 2), padding="valid")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = Dropout(DropoutRatio)(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv1)
    uconv1 = residual_block(uconv1, start_neurons * 1)
    uconv1 = residual_block(uconv1, start_neurons * 1, True)
    
    output_layer_noActi = Conv2D(1, (1, 1), padding="same", activation=None)(uconv1)
    output_layer = Activation('sigmoid')(output_layer_noActi)
    
    return output_layer
    

# Get iou vector

In [9]:
def get_iou_vector(A, B):
    batch_size = A.shape[0]
    metric = []
    for batch in range(batch_size):
        t, p = A[batch]>0, B[batch]>0
        intersection = np.logical_and(t, p)
        union = np.logical_or(t, p)
        iou = (np.sum(intersection > 0) + 1e-10)/ (np.sum(union > 0) + 1e-10)
        thresholdds = np.arange(0.5, 1, 0.05)
        s = []
        for thresh in threasholds:
            s.append(iou > thresh)
        metric.append(np.mean(s))
        
    return np.mean(metric)

def my_iou_metric(label, pred):
    return tf.py_func(get_iou_vector, [label, pred>0.75], tf.float64)

In [10]:
# code download from: https://github.com/bermanmaxim/LovaszSoftmax
def lovasz_grad(gt_sorted):
    """
    Computes gradient of the Lovasz extension w.r.t sorted errors
    See Alg. 1 in paper
    """
    gts = tf.reduce_sum(gt_sorted)
    intersection = gts - tf.cumsum(gt_sorted)
    union = gts + tf.cumsum(1. - gt_sorted)
    jaccard = 1. - intersection / union
    jaccard = tf.concat((jaccard[0:1], jaccard[1:] - jaccard[:-1]), 0)
    return jaccard


# --------------------------- BINARY LOSSES ---------------------------

def lovasz_hinge(logits, labels, per_image=True, ignore=None):
    """
    Binary Lovasz hinge loss
      logits: [B, H, W] Variable, logits at each pixel (between -\infty and +\infty)
      labels: [B, H, W] Tensor, binary ground truth masks (0 or 1)
      per_image: compute the loss per image instead of per batch
      ignore: void class id
    """
    if per_image:
        def treat_image(log_lab):
            log, lab = log_lab
            log, lab = tf.expand_dims(log, 0), tf.expand_dims(lab, 0)
            log, lab = flatten_binary_scores(log, lab, ignore)
            return lovasz_hinge_flat(log, lab)
        losses = tf.map_fn(treat_image, (logits, labels), dtype=tf.float32)
        loss = tf.reduce_mean(losses)
    else:
        loss = lovasz_hinge_flat(*flatten_binary_scores(logits, labels, ignore))
    return loss


def lovasz_hinge_flat(logits, labels):
    """
    Binary Lovasz hinge loss
      logits: [P] Variable, logits at each prediction (between -\infty and +\infty)
      labels: [P] Tensor, binary ground truth labels (0 or 1)
      ignore: label to ignore
    """

    def compute_loss():
        labelsf = tf.cast(labels, logits.dtype)
        signs = 2. * labelsf - 1.
        errors = 1. - logits * tf.stop_gradient(signs)
        errors_sorted, perm = tf.nn.top_k(errors, k=tf.shape(errors)[0], name="descending_sort")
        gt_sorted = tf.gather(labelsf, perm)
        grad = lovasz_grad(gt_sorted)
        loss = tf.tensordot(tf.nn.relu(errors_sorted), tf.stop_gradient(grad), 1, name="loss_non_void")
        return loss

    # deal with the void prediction case (only void pixels)
    loss = tf.cond(tf.equal(tf.shape(logits)[0], 0),
                   lambda: tf.reduce_sum(logits) * 0.,
                   compute_loss,
                   strict=True,
                   name="loss"
                   )
    return loss


def flatten_binary_scores(scores, labels, ignore=None):
    """
    Flattens predictions in the batch (binary case)
    Remove labels equal to 'ignore'
    """
    scores = tf.reshape(scores, (-1,))
    labels = tf.reshape(labels, (-1,))
    if ignore is None:
        return scores, labels
    valid = tf.not_equal(labels, ignore)
    vscores = tf.boolean_mask(scores, valid, name='valid_scores')
    vlabels = tf.boolean_mask(labels, valid, name='valid_labels')
    return vscores, vlabels

def lovasz_loss(y_true, y_pred):
    y_true, y_pred = K.cast(K.squeeze(y_true, -1), 'int32'), K.cast(K.squeeze(y_pred, -1), 'float32')
    #logits = K.log(y_pred / (1. - y_pred))
    logits = y_pred #Jiaxin
    loss = lovasz_hinge(logits, y_true, per_image = True, ignore = None)
    return loss

## ResNet 34

In [11]:
# from https://github.com/raghakot/keras-resnet/blob/master/resnet.py
def _bn_relu(input):
    """
    Helper to build a BN -> relu block
    """
    norm = BatchNormalization(axis=CHANNEL_AXIS)(input)
    return Activation("relu")(norm)

def _conv_bn_relu(**conv_params):
    """
    Helper to build a conv -> BN -> relu block
    """
    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initalizer", "he_normal")
    padding = conv_params.setdefault('padding', 'same')
    kernel_regularizer = conv_params.setdefault('kernel_regularizer', l2(1.e-4))
    
    def f(input):
        conv = Conv2D(filters=filters, kernel_size=kernel_size,
                     strides=strides, padding=padding,
                     kernel_initializer=kernel_initializer,
                     kernel_regularizer=kernel_regularizer)(input)
        return _bn_relu(conv)
    
    return f

def _bn_relu_conv(**conv_params):
    """
    Helper to build a BN -> relu -> conv block.
    This is an improved schema proposed in http://arxiv.org/pdf/1603.05027v2.pdf
    """
    filters = conv_params['filters']
    kernel_size = conv_params['kernel_size']
    strides = conv_params.setdefault('strides', (1, 1))
    kernel_initializer = conv_params.setdefault('kernel_initializer', 'he_normal')
    padding = conv_params.setdefault('padding', 'same')
    kernel_regularizer = conv_params.setdefault('kernel_regularizer', l2(1.e-4))
    
    def f(input):
        activation = _bn_relu(input)
        return Conv2D(filters=filters, kernel_size=kernel_size,
                     strides=strides, padding=padding,
                     kernel_initializer=kernel_initializer,
                     kernel_regularizer=kernel_regularizer)(activation)
    return f

def _shortcut(input, residual):
    """
    Adds a shortcut between input and residual blcok and merges them with 'sum'
    """
    # Expand channels of shortcut to match residual
    # Stride appropriately to match residual (width, height)
    # Should be int if network architecture is correctly configured
    input_shape = K.int_shape(input)
    residual_shape = K.int_shape(residual)
    stride_width = int(round(input_shape[ROW_AXIS] / residual_shape[ROW_AXIS]))
    stride_height = int(round(input_shape[COL_AXIS] / residual_shape[COL_AXIS]))
    equal_channels = input_shape[CHANNEL_AXIS] == residual_shape[CHANNEL_AXIS]
    
    shortcut = input
    if stride_width > 1 or stride_height > 1 or not equal_channels:
        shortcut = Conv2D(filters=residual_shape[CHANNEL_AXIS],
                         kernel_size=(1, 1),
                         strides=(stride_width, stride_height),
                         padding='valid',
                         kernel_initializer='he_normal',
                         kernel_regularizer=l2(0.0001))(input)

    return add([shortcut, residual])
    
def basic_block(filters, init_strides=(1, 1), is_first_block_of_first_layer=False):
    """
    Basic 3 X 3 convolution blocks for use on resnets with layers <= 34
    """
    def f(input):
        
        if is_first_block_of_first_layer:
            # don't repeat bn ->relu since we just did bn -> relu -> maxpool
            conv1 = Conv2D(filters=filters, kernel_size=(3, 3),
                          strides=init_strides,
                          padding='same',
                          kernel_initializer='he_normal',
                          kernel_regularizer=l2(1e-4))(input)
        else:
            conv1 = _bn_relu_conv(filters=filters, kernel_size=(3, 3),
                                 strides=init_strides)(input)
        
        residual = _bn_relu_conv(filters=filters, kernel_size=(3,3))(conv1)
        return _shortcut(input, residual)
    
    return f

def _residual_block(block_function, filters, repetitions, is_first_layer=False):
    """
    Builds a residual block with repeating bottleneck blocks
    """
    def f(input):
        for i in range(repetitions):
            init_strides = (1, 1)
            if i == 0 and not is_first_layer:
                init_strides = (2, 2)

            input = block_function(filters=filters, init_strides=init_strides,
                                  is_first_block_of_first_layer=(is_first_layer and i == 0))(input)
            
        return input
    return f

def _handle_dim_ordering():
    global ROW_AXIS
    global COL_AXIS
    global CHANNEL_AXIS
    if K.image_dim_ordering() == 'tf':
        ROW_AXIS = 1
        COL_AXIS = 2
        CHANNEL_AXIS = 3
    else:
        CHANNEL_AXIS =1
        ROW_AXIS = 2
        COL_AXIS = 3
        
def _get_block(identifier):
    if isinstance(identifier, six.string_types):
        res = globals().get(identifier)
        if not res:
            raise ValueError('Invalid {}'.format(identifer))
        return res
    return identifier

class ResnetBuilder(object):
    @staticmethod
    def build(input_shape, block_fn, repetitions, input_tensor):
        _handle_dim_ordering()
        if len(input_shape) != 3:
            raise Exception('Input shape should be a tuple (nb_channels, nb_rows, nb_cols)')
            
        # permute dimention order if necessary-> this is so comfusing.
#         if K.image_dim_ordering() == 'tf':
#             input_shape = (input_shape[1], input_shape[2], input_shape[0])
        
        # Load function from str if needed.
        block_fn = _get_block(block_fn)
        
        if input_tensor is None:
            img_input = Input(shape=input_shape)
        else:
            if not K.is_keras_tensor(input_tensor):
                img_input = Input(tensor=input_tensor, shape=input_shape)
            else:
                img_input = input_tensor
                
        conv1 = _conv_bn_relu(filters=64, kernel_size=(7, 7), strides=(2, 2))(img_input)
        pool1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(conv1)
        
        block = pool1
        filters = 64
        for i, r in enumerate(repetitions):
            block = _residual_block(block_fn, filters=filters, repetitions=r, is_first_layer=(i==0))(block)
            filters *= 2
        # Last activation
        block = _bn_relu(block)
        
        model = Model(inputs=img_input, outputs=block)
        return model
        
    @staticmethod
    def build_resnet_34(input_shape, input_tensor):
        return ResnetBuilder.build(input_shape, basic_block, [3, 4, 6, 3], input_tensor)


## U-Net with ResNet34 Encoder

In [12]:
def UResNet34(input_shape=(128, 128, 1), classes=1, decoder_filters=16, decoder_block_type='upsampling', 
             decoder_weights='imagenet', input_tensor=None, activation='sigmoid', **kwargs):
    
    backbone = ResnetBuilder.build_resnet_34(input_shape=input_shape, input_tensor=input_tensor)
    
    input_layer = backbone.input
    output_layer = build_model(input_layer, 16, 0.5)
    model = Model(input_layer, output_layer)
    c = optimizers.adam(lr = 0.01)
    
    model.compile(loss='binary_crossentropy', optimizer=c, metrics=[my_iou_metric])
    model.name = 'u-resnet34'
    
    return model

In [13]:
img_size_ori = 101
img_size_target = 101
epochs = 1
batch_size = 32
# model1 = UResNet34(input_shape=(1, img_size_target, img_size_target))
model1 = UResNet34(input_shape=(img_size_target, img_size_target, 3))

## Data load and augmentation

In [14]:
master_df = pd.read_csv('./data/depth_prop_cls_bad_mask.csv')
id_depth_df = master_df[['id', 'z']].set_index('id')['z']
depth_dict = id_depth_df.to_dict()

In [15]:
train_val_df = master_df[(master_df['data_type'] == 'train')]
ids = train_val_df['id'].values
classes = train_val_df['salt_propotion_class'].values

In [16]:
train_ids, val_ids = train_test_split(ids, test_size=0.2, random_state=43, stratify=classes)

In [17]:
#make generator
batch_size = 32
tgs_generator = Generator(train_ids=train_ids, depth_dict=depth_dict, val_ids=val_ids, batch_size=batch_size, size=(img_size_target, img_size_target, 3), feature_out=False)

In [18]:
#steps_per_epoch
steps_per_epoch = len(train_ids) // batch_size
validation_steps = len(val_ids) // batch_size
print(steps_per_epoch)
print(validation_steps)

97
24


In [14]:
# b  = next(tgs_generator.generate(True))

In [15]:
# b[0].shape

## Train 1

In [ ]:
c = optimizers.adam(lr = 0.01)
reduce_lr = ReduceLROnPlateau(monitor='mean_iou_threshold', mode='max', factor=0.5, patience=5, min_lr=0.0001, verbose=1)
train_util.train(model1, 
                 tgs_generator.generate(True), 
                 tgs_generator.generate(False), 
                 metrics=[train_util.dice_coef, 'binary_accuracy', train_util.true_positive_rate, train_util.iou_coef, train_util.mean_iou_threshold],
                 epochs=3000, steps_per_epoch=steps_per_epoch, 
                 optimizer=c,
                 validation_steps=validation_steps,
                    loss='binary_crossentropy')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_27 (InputLayer)           (None, 101, 101, 3)  0                                            
__________________________________________________________________________________________________
conv2d_749 (Conv2D)             (None, 101, 101, 16) 448         input_27[0][0]                   
__________________________________________________________________________________________________
batch_normalization_705 (BatchN (None, 101, 101, 16) 64          conv2d_749[0][0]                 
__________________________________________________________________________________________________
activation_695 (Activation)     (None, 101, 101, 16) 0           batch_normalization_705[0][0]    
__________________________________________________________________________________________________
conv2d_750

Epoch 1/3000
 - 72s - loss: 0.3841 - dice_coef: 0.2577 - binary_accuracy: 0.8450 - true_positive_rate: 0.5943 - iou_coef: 0.4972 - mean_iou_threshold: 0.4229 - val_loss: 3.8268 - val_dice_coef: 0.4054 - val_binary_accuracy: 0.7626 - val_true_positive_rate: 0.0000e+00 - val_iou_coef: 0.4023 - val_mean_iou_threshold: 0.4023
Finished epoch: 0
{'dice_coef': 0.2576808273485026, 'binary_accuracy': 0.8450489879883442, 'true_positive_rate': 0.5943053347087398, 'val_mean_iou_threshold': 0.40234375, 'loss': 0.38411176189319374, 'val_true_positive_rate': 0.0, 'val_binary_accuracy': 0.7625567217667898, 'mean_iou_threshold': 0.4229381446985854, 'iou_coef': 0.4971560344253619, 'val_loss': 3.8268001675605774, 'val_iou_coef': 0.40234433859586716, 'val_dice_coef': 0.4053647431234519}

Epoch 00001: val_loss improved from inf to 3.82680, saving model to ./log/2018_1005_2350/best_weights.hdf5
Epoch 2/3000
 - 50s - loss: 0.3002 - dice_coef: 0.2868 - binary_accuracy: 0.8838 - true_positive_rate: 0.6801 - io

Finished epoch: 9
{'dice_coef': 0.35984367239721043, 'binary_accuracy': 0.9282405284262195, 'true_positive_rate': 0.8228767999668711, 'val_mean_iou_threshold': 0.696614588300387, 'loss': 0.1953351100157831, 'val_true_positive_rate': 0.8681416983405749, 'val_binary_accuracy': 0.9476639355222384, 'lr': 0.01, 'mean_iou_threshold': 0.6636597950433948, 'iou_coef': 0.7065671737661067, 'val_loss': 0.14330317607770363, 'val_iou_coef': 0.740116981168588, 'val_dice_coef': 0.4174199153979619}

Epoch 00010: val_loss improved from 0.14765 to 0.14330, saving model to ./log/2018_1005_2350/best_weights.hdf5
Epoch 11/3000
 - 43s - loss: 0.1949 - dice_coef: 0.3645 - binary_accuracy: 0.9272 - true_positive_rate: 0.8256 - iou_coef: 0.7063 - mean_iou_threshold: 0.6610 - val_loss: 0.2603 - val_dice_coef: 0.3291 - val_binary_accuracy: 0.9032 - val_true_positive_rate: 0.6057 - val_iou_coef: 0.6772 - val_mean_iou_threshold: 0.6305
Finished epoch: 10
{'dice_coef': 0.36445009892748803, 'binary_accuracy': 0.92722

 - 47s - loss: 0.1770 - dice_coef: 0.3727 - binary_accuracy: 0.9337 - true_positive_rate: 0.8386 - iou_coef: 0.7342 - mean_iou_threshold: 0.6947 - val_loss: 0.1544 - val_dice_coef: 0.4149 - val_binary_accuracy: 0.9455 - val_true_positive_rate: 0.8351 - val_iou_coef: 0.7076 - val_mean_iou_threshold: 0.6629
Finished epoch: 19
{'dice_coef': 0.37266379640888925, 'binary_accuracy': 0.9337130692816272, 'true_positive_rate': 0.8385688994348663, 'val_mean_iou_threshold': 0.6628906205296516, 'loss': 0.1770021783322403, 'val_true_positive_rate': 0.8351384922862053, 'val_binary_accuracy': 0.9454529012242953, 'lr': 0.01, 'mean_iou_threshold': 0.694716492879022, 'iou_coef': 0.734153659073348, 'val_loss': 0.15444323575745025, 'val_iou_coef': 0.7076001514991125, 'val_dice_coef': 0.41486546273032826}

Epoch 00020: val_loss did not improve
Epoch 21/3000
 - 47s - loss: 0.1711 - dice_coef: 0.3832 - binary_accuracy: 0.9359 - true_positive_rate: 0.8495 - iou_coef: 0.7300 - mean_iou_threshold: 0.6899 - val_

Finished epoch: 28
{'dice_coef': 0.40259236404576254, 'binary_accuracy': 0.9478508399933884, 'true_positive_rate': 0.8652070773016546, 'val_mean_iou_threshold': 0.7684895843267441, 'loss': 0.13492588241843834, 'val_true_positive_rate': 0.9115916267037392, 'val_binary_accuracy': 0.9600719610850016, 'lr': 0.0019999999, 'mean_iou_threshold': 0.7328608251109565, 'iou_coef': 0.7658238619873204, 'val_loss': 0.10554994239161412, 'val_iou_coef': 0.8030097136894861, 'val_dice_coef': 0.461867019534111}

Epoch 00029: val_loss improved from 0.10986 to 0.10555, saving model to ./log/2018_1005_2350/best_weights.hdf5
Epoch 30/3000
 - 47s - loss: 0.1472 - dice_coef: 0.4007 - binary_accuracy: 0.9440 - true_positive_rate: 0.8548 - iou_coef: 0.7605 - mean_iou_threshold: 0.7244 - val_loss: 0.1180 - val_dice_coef: 0.4325 - val_binary_accuracy: 0.9553 - val_true_positive_rate: 0.8639 - val_iou_coef: 0.7910 - val_mean_iou_threshold: 0.7602
Finished epoch: 29
{'dice_coef': 0.4006812656970368, 'binary_accuracy

...

 - 50s - loss: 0.1126 - dice_coef: 0.4247 - binary_accuracy: 0.9574 - true_positive_rate: 0.8883 - iou_coef: 0.7861 - mean_iou_threshold: 0.7545 - val_loss: 0.0934 - val_dice_coef: 0.4799 - val_binary_accuracy: 0.9655 - val_true_positive_rate: 0.9083 - val_iou_coef: 0.8191 - val_mean_iou_threshold: 0.7928
Finished epoch: 156
{'dice_coef': 0.42472282727969063, 'binary_accuracy': 0.9573724067088255, 'true_positive_rate': 0.8882586052737285, 'val_mean_iou_threshold': 0.7928385411699613, 'loss': 0.11258425098872676, 'val_true_positive_rate': 0.9083060224850973, 'val_binary_accuracy': 0.9655395348866781, 'lr': 2.0479998e-10, 'mean_iou_threshold': 0.7545425252816111, 'iou_coef': 0.7861080489207789, 'val_loss': 0.09339929729079206, 'val_iou_coef': 0.8191012069582939, 'val_dice_coef': 0.47992218534151715}

Epoch 00157: val_loss did not improve
Epoch 158/3000
 - 51s - loss: 0.1182 - dice_coef: 0.4211 - binary_accuracy: 0.9552 - true_positive_rate: 0.8805 - iou_coef: 0.7922 - mean_iou_threshold:

 - 46s - loss: 0.1097 - dice_coef: 0.4296 - binary_accuracy: 0.9582 - true_positive_rate: 0.8916 - iou_coef: 0.7934 - mean_iou_threshold: 0.7616 - val_loss: 0.0933 - val_dice_coef: 0.4786 - val_binary_accuracy: 0.9656 - val_true_positive_rate: 0.9078 - val_iou_coef: 0.8186 - val_mean_iou_threshold: 0.7917
Finished epoch: 166
{'dice_coef': 0.42963596901942774, 'binary_accuracy': 0.9582129557108142, 'true_positive_rate': 0.8915604113303509, 'val_mean_iou_threshold': 0.7916666666666666, 'loss': 0.10971975015457143, 'val_true_positive_rate': 0.9077988912661871, 'val_binary_accuracy': 0.9656427924831709, 'lr': 4.0959996e-11, 'mean_iou_threshold': 0.7615979363008872, 'iou_coef': 0.7933936739705273, 'val_loss': 0.09330211222792666, 'val_iou_coef': 0.8185697918136915, 'val_dice_coef': 0.4786233032743136}

Epoch 00167: val_loss did not improve
Epoch 168/3000
 - 46s - loss: 0.1152 - dice_coef: 0.4237 - binary_accuracy: 0.9576 - true_positive_rate: 0.8942 - iou_coef: 0.7873 - mean_iou_threshold: 

 - 47s - loss: 0.1197 - dice_coef: 0.4222 - binary_accuracy: 0.9550 - true_positive_rate: 0.8788 - iou_coef: 0.7919 - mean_iou_threshold: 0.7609 - val_loss: 0.0935 - val_dice_coef: 0.4795 - val_binary_accuracy: 0.9658 - val_true_positive_rate: 0.9072 - val_iou_coef: 0.8183 - val_mean_iou_threshold: 0.7914
Finished epoch: 176
{'dice_coef': 0.42220170688383357, 'binary_accuracy': 0.9550477137270662, 'true_positive_rate': 0.8788462692929297, 'val_mean_iou_threshold': 0.7914062490065893, 'loss': 0.1196692362811762, 'val_true_positive_rate': 0.9072264457742373, 'val_binary_accuracy': 0.9658149878184, 'lr': 8.191999e-12, 'mean_iou_threshold': 0.7608569583941981, 'iou_coef': 0.7918545400973448, 'val_loss': 0.09347581894447406, 'val_iou_coef': 0.8183150415619215, 'val_dice_coef': 0.47954122970501584}

Epoch 00177: val_loss did not improve
Epoch 178/3000
 - 44s - loss: 0.1157 - dice_coef: 0.4253 - binary_accuracy: 0.9564 - true_positive_rate: 0.8873 - iou_coef: 0.7904 - mean_iou_threshold: 0.75

 - 50s - loss: 0.1140 - dice_coef: 0.4273 - binary_accuracy: 0.9562 - true_positive_rate: 0.8808 - iou_coef: 0.7965 - mean_iou_threshold: 0.7648 - val_loss: 0.0935 - val_dice_coef: 0.4798 - val_binary_accuracy: 0.9657 - val_true_positive_rate: 0.9068 - val_iou_coef: 0.8183 - val_mean_iou_threshold: 0.7914
Finished epoch: 186
{'dice_coef': 0.42725179782233286, 'binary_accuracy': 0.9561816841056666, 'true_positive_rate': 0.8808288322281592, 'val_mean_iou_threshold': 0.7914062514901161, 'loss': 0.11397106296469256, 'val_true_positive_rate': 0.906825602054596, 'val_binary_accuracy': 0.9657452876369158, 'lr': 1.6383998e-12, 'mean_iou_threshold': 0.7647551543933829, 'iou_coef': 0.7964941945272622, 'val_loss': 0.09349617315456271, 'val_iou_coef': 0.818279559413592, 'val_dice_coef': 0.4798012003302574}

Epoch 00187: val_loss did not improve
Epoch 188/3000
 - 46s - loss: 0.1177 - dice_coef: 0.4214 - binary_accuracy: 0.9557 - true_positive_rate: 0.8801 - iou_coef: 0.7872 - mean_iou_threshold: 0.

 - 51s - loss: 0.1149 - dice_coef: 0.4275 - binary_accuracy: 0.9562 - true_positive_rate: 0.8915 - iou_coef: 0.7930 - mean_iou_threshold: 0.7626 - val_loss: 0.0935 - val_dice_coef: 0.4800 - val_binary_accuracy: 0.9656 - val_true_positive_rate: 0.9074 - val_iou_coef: 0.8187 - val_mean_iou_threshold: 0.7922
Finished epoch: 196
{'dice_coef': 0.4274896312620222, 'binary_accuracy': 0.956176850599112, 'true_positive_rate': 0.8914659932716605, 'val_mean_iou_threshold': 0.7921874995032946, 'loss': 0.11486184923458345, 'val_true_positive_rate': 0.9074364279707273, 'val_binary_accuracy': 0.9656387070814768, 'lr': 3.2767997e-13, 'mean_iou_threshold': 0.7626288662251738, 'iou_coef': 0.7929611273647583, 'val_loss': 0.09345803782343864, 'val_iou_coef': 0.8187409887711207, 'val_dice_coef': 0.47995466614762944}

Epoch 00197: val_loss did not improve
Epoch 198/3000
 - 45s - loss: 0.1128 - dice_coef: 0.4281 - binary_accuracy: 0.9577 - true_positive_rate: 0.8926 - iou_coef: 0.7974 - mean_iou_threshold: 0

## Train 2

In [19]:
model1 = load_model('./log/2018_1005_2350/best_weights.hdf5', custom_objects={
#     'mean_iou': train_util.mean_iou, 
    'dice_p_bce':train_util.dice_p_bce, 
    'dice_coef':train_util.dice_coef,
    'true_positive_rate':train_util.true_positive_rate,
    'iou_coef':train_util.iou_coef,
    'mean_iou_threshold':train_util.mean_iou_threshold
})

input_x = model1.layers[0].input

output_layer = model1.layers[-1].input
model = Model(input_x, output_layer)
c = optimizers.adam(lr = 0.01)

# lovasz_loss need input range (-~, +~), so cancel the last "sigmoid" activation
#Then the default threshold for pixel prediction is 0 instead of 0.5 as in my_iou_metric_2.


In [20]:
model.compile(loss=lovasz_loss, optimizer=c, metrics=[my_iou_metric])

TypeError: 'NoneType' object cannot be interpreted as an integer

In [18]:
tf.__version__

'1.4.0'

In [5]:
pip install keras --upgrade

SyntaxError: invalid syntax (<ipython-input-5-430f2f0324f7>, line 1)

In [16]:
c = optimizers.adam(lr = 0.01)
reduce_lr = ReduceLROnPlateau(monitor='val_mean_iou_threshold', mode='max', factor=0.5, patience=5, min_lr=0.0001, verbose=1)
train_util.train(model1, 
                 tgs_generator.generate(True), 
                 tgs_generator.generate(False), 
                 metrics=[train_util.dice_coef, 'binary_accuracy', train_util.true_positive_rate, train_util.iou_coef, train_util.mean_iou_threshold],
                 epochs=3000, steps_per_epoch=steps_per_epoch, 
                 optimizer=c,
                 validation_steps=validation_steps,
                loss=lovasz_loss)

TypeError: 'NoneType' object cannot be interpreted as an integer